In [1]:
%%capture
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain ctransformers sentence-transformers pandas nltk spacy PyPDF langchain-chroma
%pip install --upgrade --quiet sentence-transformers langchain-chroma langchain langchain-openai > /dev/null


In [2]:
# connecting to database
path="../data"

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
# Load documents from PDF
loader = DirectoryLoader(path, glob="disease-handbook-complete.pdf", loader_cls=PyPDFLoader, show_progress=True)
documents = loader.load()

100%|██████████| 1/1 [00:03<00:00,  3.50s/it]


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Split the preprocessed documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

Split 86 documents into 260 chunks.


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
#embedding model
def embedding_model():
    # embedding = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
    embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
    return embedding
embedding = embedding_model()
embedding

c:\Users\somes\venv-jupyterenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-large-en-v1.5', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [5]:
from langchain_chroma import Chroma
# Creating vector db to store Embeddings
# persist_directory = "./drive/MyDrive/Colab Notebooks/nlp/chroma_db-v2"
db = Chroma.from_documents(chunks, embedding, persist_directory="../chroma_db")

In [6]:
from langchain import PromptTemplate
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [7]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [8]:
query = "What are Allergies"

# docs=db.similarity_search(query, k=3)

docs=db.similarity_search_with_relevance_scores(query, k=7)
print("Result", docs)

Result [(Document(page_content='People react to direct contact from the plant or from indirect contact from clothing, or other objects contaminated from plant contact.  (A family pet can also indirectly pass this to people when its fur is contaminated).  It is best to consult a physician for treatment.  \n \nHives is a rash that may happen when a person is hypersensitive to such things as certain foods, \ndrugs, and bee stings.  It may also be due to emotional factors.  T he rash is usually itchy, raised, reddish \nwelts on the skin.  Hives that are accompanied by difficulty breathing, unusual anxiety and hives occurring all over the body needs to be seen by a physician immediately.  \n \nAnother common rash experienced by childre n during the summer months is known as Swimmer’s \nItch.  It is a form of dermatitis (i.e., inflammation of the skin) that is caused by larvae of certain worms when they attempt to penetrate the skin.  This results in a mild allergic reaction.  The worms tha 

In [10]:
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
# importing model
# llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",
#                   model_type="llama",
#                   config={'max_new_tokens':512,
#                           'temperature':0.8})

import transformers
import torch

model_id = "aaditya/OpenBioLLM-Llama3-70B"

llm = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="auto",
)


qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

c:\Users\somes\venv-jupyterenv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\somes\.cache\huggingface\hub\models--aaditya--OpenBioLLM-Llama3-70B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [12]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

c:\Users\somes\venv-jupyterenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of tokens (619) exceeded maximum context length (512).
Number of tokens (620) exceeded maximum context length (512).
Number of tokens (621) exceeded maximum context length (512).
Number of tokens (622) exceeded maximum context length (512).
Number of tokens (623) exceeded maximum context length (512).
Number of tokens (624) exceeded maximum context length (512).
Number of tokens (625) exceeded maximum context length (512).
Number of tokens (626) exceeded maximum context length (512).
Number of tokens (627) exceeded maximum context length (512).
Number of tokens (628) exceeded maximum context length (512).
Number of tokens (629) exceeded maximum context length (512).
Number of tokens (630) exceeded maximum context length (512).
Number 

Response :  Symptomit is not applicable to provide the above  
The symptoms  The rotavirus usually the question three; However the exclusion of rotavirus is not applicable to make up to help!  Because they can only if your child' 1, The specific excludes illness. 
You are all the symptoms. Your healthcaregive  The disease, rotate
If you may also depends on  It is there is not applicable to determine if your child’susual   
No, none of rotavirus is not applicable for rotavirus is a rotavirus diagnosis is it depends on page 
It is the information. The rotavirus is not provided in this handwides when it does not specified. If your childcaregive symptoms are not applicable to be sure! 
The following information about 
Symptacet if you do not applicable to treat rotavirus is not applicable to help the healthcarefully understand that depends on page  The most of rotavirus may befor the following pieces of rotavirus is not applicable to make their observations show no   10 Rotavirus may also 